<a href="https://colab.research.google.com/github/priyankari0999/Mayo/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

dataset = pd.read_csv("/content/PREPROCESSED-NOTES.csv")

text_data = dataset["text"].to_list()

sdoh_data = {
    "sdoh_community_present": dataset["sdoh_community_present"].to_list(),
    "sdoh_community_absent": dataset["sdoh_community_absent"].to_list(),
    "sdoh_education": dataset["sdoh_education"].to_list(),
    "sdoh_economics": dataset["sdoh_economics"].to_list(),
    "sdoh_environment": dataset["sdoh_environment"].to_list(),
    "behavior_alcohol": dataset["behavior_alcohol"].to_list(),
    "behavior_tobacco": dataset["behavior_tobacco"].to_list(),
    "behavior_drug": dataset["behavior_drug"].to_list()
}

base_path = 'test_train_split/behavior_drug'
os.makedirs(base_path, exist_ok=True)

# Iterate through each SDOH data category
for category, data in sdoh_data.items():
    base_path = f"test_train_split/{category}"
    os.makedirs(base_path, exist_ok=True)

    # Split data for the current category
    X_train, X_val, y_train, y_val = train_test_split(
        text_data, data, random_state=0, train_size=0.8, stratify=data
    ) #maybe try a different test size (0.7/0.3)

        # Save all splits as CSV files
    pd.DataFrame({"text": X_train}).to_csv(f"{base_path}/X_train.csv", index=False)
    pd.DataFrame({"text": X_val}).to_csv(f"{base_path}/X_val.csv", index=False)
    pd.DataFrame({category: y_train}).to_csv(f"{base_path}/y_train.csv", index=False)
    pd.DataFrame({category: y_val}).to_csv(f"{base_path}/y_val.csv", index=False)

'''
def compute_metrics(eval_pred):
  labels = eval_pred.label_ids
  preds = eval_pred.predictions.argmax(-1)
  precision = precision_score(labels, preds, average='weighted')
  recall = recall_score(labels, preds, average='weighted')
  f1 = f1_score(labels, preds, average='weighted')
  acc = accuracy_score(labels, preds)
  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
      }

'''

print("Hello")

Hello


In [2]:
import datetime
import os
import pandas as pd
import json
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt

def save_metrics_to_csv(json_filepath, csv_filename):
    with open(json_filepath) as file:
        data = json.load(file)

        log_history = data['log_history'] #focus on this column for history
        df = pd.DataFrame(log_history) # Convert the list of dictionaries to a DataFrame

        df.to_csv(csv_filename, index=False)
'''
# Function to plot metrics
def plot_metrics_from_csv(csv_filepath, output_dir):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_filepath)

    plt.figure(figsize=(10, 6))

    # Plot accuracy
    plt.subplot(2, 2, 1)
    plt.plot(df['epoch'], df['eval_accuracy'], label='Accuracy')
    plt.title('Accuracy')
    plt.xlabel('epoch')
    plt.legend()

    # Plot precision
    plt.subplot(2, 2, 2)
    plt.plot(df['epoch'], df['eval_precision'], label='Precision')
    plt.title('Precision')
    plt.xlabel('epoch')
    plt.legend()

    # Plot recall
    plt.subplot(2, 2, 3)
    plt.plot(df['epoch'], df['eval_recall'], label='Recall')
    plt.title('Recall')
    plt.xlabel('epoch')
    plt.legend()

    # Plot F1 score
    plt.subplot(2, 2, 4)
    plt.plot(df['epoch'], df['eval_f1'], label='F1 Score')
    plt.title('F1 Score')
    plt.xlabel('epoch')
    plt.legend()

    plt.tight_layout()

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    plt.savefig(os.path.join(output_dir, 'metrics_plot.png'))
    plt.show()
'''
def plot_metric_from_tensor(log_dir, output_dir):

    event_acc = event_accumulator.EventAccumulator(log_dir)
    event_acc.Reload()

    graph1_data = event_acc.Scalars("eval/loss")
    graph2_data = event_acc.Scalars("train/loss")

    # Access step and value directly from events
    steps1 = [event.step for event in graph1_data]
    values1 = [event.value for event in graph1_data]

    steps2 = [event.step for event in graph2_data]
    values2 = [event.value for event in graph2_data]

    plt.figure(figsize=(10, 6))
    plt.plot(steps1, values1, label="Eval Loss")
    plt.plot(steps2, values2, label="Train Loss")
    plt.legend()
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("Overlap")
    # plt.show()


    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Save the graph to the specified folder
    plt.savefig(os.path.join(output_dir, 'metrics_plot.png'))
    plt.show()

def compute_metrics(pred): #evaluation metrics function
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def get_latest_checkpoint(folder_path):
    # Get a list of all files and directories in the specified folder
    files_and_dirs = os.listdir(folder_path)

    # Filter only directories (assumed to be checkpoints)
    checkpoint_dirs = [d for d in files_and_dirs if os.path.isdir(os.path.join(folder_path, d))]

    if not checkpoint_dirs:
        print("No checkpoint directories found.")
        return None

    # Extract the checkpoint numbers from the directory names
    checkpoint_numbers = [int(d.split('-')[1]) for d in checkpoint_dirs]

    # Identify the directory with the highest checkpoint number
    latest_checkpoint = os.path.join(folder_path, f"checkpoint-{max(checkpoint_numbers)}")

    return latest_checkpoint

In [ ]:
!pip install transformers[torch] accelerate -U plotting
import datetime
import os
import torch
from torch.optim import AdamW  # variant of Adam with weight decay
from torch.utils.data import Dataset
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, TrainingArguments, Trainer
import pandas as pd
import json
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup

from plotting import *

# Initialize tokenizer, this is standard approach with GPT-2
# Loading GPT-2 tokenizer and model for sequence classification
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", num_labels = 5)
# GPT-2 uses the same token for end-of-sentence and padding.
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

# Ensure the model is compatible with the tokenizer settings
configuration = GPT2ForSequenceClassification.config_class.from_pretrained("gpt2", num_labels= 5)
configuration.pad_token_id = tokenizer.pad_token_id
model = GPT2ForSequenceClassification(configuration)
# model = GPT2ForSequenceClassification.from_pretrained("gpt2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Set up no decay for certain model parameters to avoid regularization on them
no_decay = ['bias', 'LayerNorm.weight']  # weight decay with a minor penalty during
optimizer_grouped_parameters = [  # no selects params added
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)
dataset = pd.read_csv("/content/PREPROCESSED-NOTES.csv")

text_data = dataset["text"].to_list()
sdoh_data = dataset["behavior_alcohol"].to_list()

timestamp_fortrain = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(text_data, sdoh_data, random_state=0, train_size=0.8,
                                                  stratify=sdoh_data) #make it test = 0.3
max_seq_length = 100  # actually 50 but increase to accomadate outliers

# Calculate the number of trainable parameters in the model
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
model_size_MB = num_trainable_params * 4 / (1024 ** 2)
effective_batch = 8 / (50*4*model_size_MB) #gpu/seqlength * 4 * model size

train_encodings = tokenizer(X_train, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors='pt')
val_encodings = tokenizer(X_val, truncation=True, padding='max_length', max_length=max_seq_length, return_tensors='pt')


# custom Dataset class for loading training and validation data
class DataLoader(Dataset):
    def __init__(self, encodings, labels):
        """ :rtype: object """
        self.encodings = encodings
        self.labels = torch.tensor(labels, dtype=torch.long)  # Converting to tensor , maybe use just 'labels'

    def __getitem__(self, idx):
        try:
            item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
            item['labels'] = self.labels[idx].clone().detach()  # Already a tensor, just clone and detach
            return item
        except Exception as e:
            print(f"index error: {idx}: {e}")
            return None

    def __len__(self):
        return len(self.labels) # detach from tensor device

# Initialize the DataLoader for training and validation sets with the tokenized encodings
train_dataset: DataLoader = DataLoader(
    train_encodings,  # These should be the output from the tokenizer
    y_train  # These should be your labels, as a list or tensor
)

val_dataset = DataLoader(
    val_encodings,  # These should be the output from the tokenizer
    y_val  # These should be your labels, as a list or tensor
)

tensor_logs = f'./logs/tensor_logs/{timestamp_fortrain}' #create seperate logs for tensor/epoch
os.makedirs(tensor_logs, exist_ok=True)
epoch_logs = f'./logs/epoch_logs/{timestamp_fortrain}'
os.makedirs(epoch_logs, exist_ok=True)

# training args - need to adjust
training_args = TrainingArguments(
    output_dir= epoch_logs,  # change to epoch log directory, convert to a text
    logging_strategy='epoch',  # characterize as epoch
    num_train_epochs=10,
    #per_device_train_batch_size=64,  # cpu constraint,  64 approp
    per_device_train_batch_size=16, #reduced batch sie
    per_device_eval_batch_size=64,  # gradient accum if batch size of two, 64 approp
    save_strategy= 'epoch',
    warmup_steps=500,
    weight_decay=1e-5,
    logging_dir= tensor_logs,  # change to tensor logs
    eval_steps=100,
    evaluation_strategy="epoch",
    #accumulate gradients over 4 steps
    #gradient_accumulation_steps = 4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
evaluation_results = trainer.evaluate()

#readable results
latest_checkpoint = get_latest_checkpoint(epoch_logs) # latest checkpoint update to csv
json_path = os.path.join(latest_checkpoint, 'trainer_state.json')
save_metrics_to_csv(json_path, 'eval_metric.csv') #update metrics
plot_metric_from_tensor(tensor_logs, 'graphs')

save_directory = "saved_models/gpt2"

os.makedirs(save_directory, exist_ok=True)
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print("Evaluation Results:", evaluation_results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.306000,0.911646,0.664769,0.632532,0.632134,0.664769
2,0.742500,0.639250,0.782206,0.747326,0.782850,0.782206


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
